In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
train = pd.read_csv("train.csv")
leaderboard = pd.read_csv("leaderboard_dataset.csv")
test = pd.read_csv("test.csv")

In [3]:
y = train["VAR21"]
X = train.drop(["VAR21"], axis=1)

In [4]:
from sklearn.preprocessing import LabelEncoder

for c in X.columns:
    if X[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(X[c].values) + list(test[c].values) + list(leaderboard[c].values)) 
        X[c] = lbl.transform(list(X[c].values))
        test[c] = lbl.transform(list(test[c].values))
        leaderboard[c] = lbl.transform(list(leaderboard[c].values))

In [5]:
lbl = LabelEncoder() 
lbl.fit(list(y.values)) 
y = lbl.transform(list(y))

In [6]:
y

array([1, 0, 2, ..., 1, 1, 0])

In [8]:
import xgboost as xgb

xgb_params = {
    'n_trees': 1000, 
    'eta': 0.005,
    'max_depth': 6,
    'subsample': 0.93,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'silent': 1,
    'num_class' :3
}

from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer

dtrain = xgb.DMatrix(X.drop(["VAR1"], axis=1), y)
dtest = xgb.DMatrix(leaderboard.drop(["VAR1"], axis=1))

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=1500)


# check f2-score (to get higher score - increase num_boost_round in previous cell)

# make predictions and save results
y_preds = model.predict(dtest)

In [17]:
y_mod = y_preds
a = []


for i in y_mod:
    if i[0]>i[1]-0.11 and i[0]>i[2]-0.1:
        a.append(0)
    elif i[1]>i[0]+0.11 and i[1]>i[2]-0.1:
        a.append(1)
    else:
        a.append(2)
        
d = {'col1': leaderboard["VAR1"], 'col2': [int(i) for i in a]}
df = pd.DataFrame(data=d)
df["col2"][df["col2"] == 0] = "High"
df["col2"][df["col2"] == 1] = "Low"
df["col2"][df["col2"] == 2] = "Medium"

import shutil 

filename = "Quant404_IITGuwahati_59"
df.to_csv(filename+'.csv', index=False, header=False)
shutil.copyfile("Quant404_IITGuwahati_54.ipynb", filename+".ipynb")

'Quant404_IITGuwahati_59.ipynb'